# Practical 4: National Environmental Footprints

In this practical, you will calculate the total environmental footprints of countries. 

The objectives of the practical are:
- To understand the structure of the MR IO system
- Calculate national footprints
- Gain transferable skills:
    - Master slicing

## Exercise 1: Load the data and labels

Import the IO data. The dataset is constructed from real-world data from the year of 2015.

In [255]:
# Import modules
import pandas as pd
import numpy as np

idx = pd.IndexSlice

#### 1.1 Import the IO data

In [134]:
# Import Z, Y, V, F
Z = pd.read_csv("data/Z.txt", delimiter="\t", header=None)
Y = pd.read_csv('data/Y.txt', delimiter='\t', header=None)
V = pd.read_csv('data/V.txt', delimiter='\t', header=None)
F = pd.read_csv('data/F.txt', delimiter='\t', header=None)
F_y = pd.read_csv('data/F_y.txt', delimiter='\t', header=None)

#### 1.2 Import and organize your labels

In [135]:
# Import the labels.csv file from the data folder 
labels = pd.read_csv("data/labels/labels.csv", delimiter=",")
labels

,region_name,sector_category,final_demand_category,value_added_category,extension_name
0,OECD,Food,Final consumption expenditure by household,value_added,CO2 emissions (unit: tonnes/year)
1,BRICS,Clothing,Final consumption expenditure by NPISHs,NaN,Blue water consumption (unit: million m3/year)
2,ROW,Shelter,Final consumption expenditure by government,NaN,Employment (unit: 1000 people/year)
3,NaN,Construction,Gross capital formation,NaN,NaN
4,NaN,Manufactured products,NaN,NaN,NaN
5,NaN,Mobility,NaN,NaN,NaN
6,NaN,Trade,NaN,NaN,NaN
7,NaN,Services,NaN,NaN,NaN


In [192]:
# Clean labels for value added and for the extensions
va_lb = labels.value_added_category.dropna()
y_lb = labels.final_demand_category.dropna()
f_lb = labels.extension_name.dropna()
regions = labels.region_name.dropna()

In [137]:
# Import the multi_reg_sectors.csv file from the data folder 
mr_sec_labels = pd.read_csv("data/labels/multi_reg_sectors.csv", delimiter=",")
mr_sec_labels

,region,sector
0,OECD,Food
1,OECD,Clothing
2,OECD,Shelter
3,OECD,Construction
4,OECD,Manufactured products
5,OECD,Mobility
6,OECD,Trade
7,OECD,Services
8,BRICS,Food
9,BRICS,Clothing


In [138]:
# Import the multi_reg_final_demand.csv file from the data folder 
mr_fd_labels = pd.read_csv("data/labels/multi_reg_final_demand.csv", delimiter=",")
mr_fd_labels

,region,final_demand_category
0,OECD,Final consumption expenditure by household
1,OECD,Final consumption expenditure by NPISHs
2,OECD,Final consumption expenditure by government
3,OECD,Gross capital formation
4,BRICS,Final consumption expenditure by household
5,BRICS,Final consumption expenditure by NPISHs
6,BRICS,Final consumption expenditure by government
7,BRICS,Gross capital formation
8,ROW,Final consumption expenditure by household
9,ROW,Final consumption expenditure by NPISHs


#### 1.3 Assign the labels to Z to see how the data is structured

tip:  

<code>Z.columns = product_labels</code>

<code>Z.index = product_labels</code>

In [223]:
Z.columns = Z.index = mr_sec_labels

However, applying labels in that way makes it difficult to select data according to their labels.

**We need multi-indexing**

#### 1.4 Brief intro to multi-indexing
Multi-indexes are hierarchical indexes useful in the grouping and slicing of columns and rows according to given common labels/features

In [225]:
# First we assign the multiindexes to Z
Z.columns = Z.index = pd.MultiIndex.from_frame(mr_sec_labels)

Y.columns = pd.MultiIndex.from_product([regions, y_lb])
Y.index = pd.MultiIndex.from_frame(mr_sec_labels)

V.columns = pd.MultiIndex.from_frame(mr_sec_labels)
V.index = va_lb

F.columns = pd.MultiIndex.from_frame(mr_sec_labels)
F.index = f_lb

#### 1.5 You can slice dataframes with multi-indexes by using 

<code>Z.loc[pd.IndexSlice[first_level, second_level],:]</code>


In [226]:
Z.loc[pd.IndexSlice["OECD",:],pd.IndexSlice[:,["Food","Shelter"]]].sort_index(axis=1,level=0)

region                               BRICS                       OECD  \
sector                                Food       Shelter         Food   
region sector                                                           
OECD   Food                   22643.474000   3962.018900  900389.9700   
       Clothing                  30.788548     81.461491    1650.2952   
       Shelter                 1025.235800   3325.086500   60010.3940   
       Construction             199.006650    197.980180   25485.0230   
       Manufactured products  10905.157000  10785.613000  158978.6000   
       Mobility                3452.037700  12369.453000  162815.5100   
       Trade                   2761.832300   1896.825400  254235.1700   
       Services               11293.723000   8610.215900  510136.0100   

region                                             ROW              
sector                             Shelter        Food     Shelter  
region sector                                                       
OECD   Food                     2854.39390  54038.6750   2841.9367  
       Clothing                  684.95479   1028.1660    623.6473  
       Shelter                389328.34000   1595.5398   8532.8224  
       Construction            58962.17100   1216.7844   1188.5423  
       Manufactured products   89108.88200  27862.4580  25074.6230  
       Mobility               107856.19000   9980.1168  10624.8900  
       Trade                  115221.04000   3077.7672   2034.8295  
       Services               283365.99000   5484.3791  12847.6460

## Exercise 2: Calculate your IO system 

#### 2.1 First convert everything to numpy objects
It will facilitate calculations

In [129]:
#Z = Z.values
#Y = Y.values
#V = V.values
#F = F.values
#F_y = F_y.values

#### 2.2 Calculate your total product inputs and outputs and check that they match

In [227]:
x_out = Z.sum(axis=1)+Y.sum(axis=1)
x_in = Z.sum(axis=0)+V.sum(axis=0)

In [228]:
# Check if X_out and X_in are the same 
print(f"The inputs equal the outputs: {np.allclose(x_in, x_out)}")

The inputs equal the outputs: True


#### 2.3 Calculate the Leontief inverse of quantity model
Calculate the Leontief Inverse matrix: 

$L=(I-A)^{-1}$

(Note: calculate the Technical Coefficient Matrix A first, the Z matrix normalised by output.)

In [236]:
# Calculate the technical coefficient matrix A
inv_diag_x = np.diag(1/x_out)
A = Z.values @ inv_diag_x

# Make an identity matrix of the same order of A
I = np.identity(A.shape[0])


# Calculate the Leontief inverse
L = np.linalg.inv(I - A)

#### 2.4 Verify the correctness of the product output

In [239]:
# Calculate the total product output
x = L @ Y.sum(axis=1)

# Check if X is the same as X_out
print(f"The calculated X is right : {np.allclose(x, x_out)}")

The calculated X is right : True


## Exercise 3: Create a modified final demand matrix

In this exercise you will learn the two methods of slicing and isolating data in matrices

Prepare your regional final demand vectors

Two programming approaches

- Mathematical approach: Isolate the consumption of a sigle country using matrix algebra

- Slicing approach: Isolate the consumption of a single country using slicing methods from numpy and pandas

#### 3.1 Let's first assign labels to your Y matrix so that it's easiery to analyse results

In [270]:
# Turn your Y into a dataframe with labels to facilitate analysis
Y = pd.DataFrame(Y, index=pd.MultiIndex.from_frame(mr_sec_labels), columns=pd.MultiIndex.from_frame(mr_fd_labels))
Y

region                                                             OECD  \
final_demand_category        Final consumption expenditure by household   
region sector                                                             
OECD   Food                                                1.753360e+06   
       Clothing                                            2.884869e+05   
       Shelter                                             6.622146e+05   
       Construction                                        6.848830e+04   
       Manufactured products                               1.786986e+06   
       Mobility                                            1.157830e+06   
       Trade                                               4.004337e+05   
       Services                                            1.070068e+07   
BRICS  Food                                                2.223431e+04   
       Clothing                                            1.077132e+05   
       Shelter                                             1.453606e+04   
       Construction                                        3.443303e+03   
       Manufactured products                               1.644037e+05   
       Mobility                                            1.929819e+04   
       Trade                                               1.828862e+02   
       Services                                            2.200201e+04   
ROW    Food                                                1.361017e+05   
       Clothing                                            7.658174e+04   
       Shelter                                             2.659790e+04   
       Construction                                        5.852935e+03   
       Manufactured products                               2.829310e+05   
       Mobility                                            4.871657e+04   
       Trade                                               5.448228e+02   
       Services                                            8.166793e+04   

region                                                                \
final_demand_category        Final consumption expenditure by NPISHs   
region sector                                                          
OECD   Food                                             1.610136e+03   
       Clothing                                         1.290528e+01   
       Shelter                                          1.939060e+03   
       Construction                                     1.349671e+02   
       Manufactured products                            4.629975e+03   
       Mobility                                         9.812754e+03   
       Trade                                            2.697264e+03   
       Services                                         2.968987e+06   
BRICS  Food                                             1.416137e+01   
       Clothing                                         2.482360e-04   
       Shelter                                          2.450880e+00   
       Construction                                     1.208981e-02   
       Manufactured products                            4.023717e+02   
       Mobility                                         2.489097e+01   
       Trade                                            3.124122e+02   
       Services                                         9.771105e+02   
ROW    Food                                             1.065831e+02   
       Clothing                                         7.118259e-03   
       Shelter                                          2.482202e+01   
       Construction                                     1.707257e-01   
       Manufactured products                            2.148106e+03   
       Mobility                                         2.460445e+02   
       Trade                                            9.219041e+02   
       Services                                         2.246062e+04   

region                       

#### 3.2 Mathematical approach
In the mathematical approach you first create a vector where you have 1s only in the position of the columns you want to isolate in the final demand matrix

$ \mathbf{Y^{reg}} = \mathbf{Y} \mathrm{\hat{i_j}^{reg}}$

In [381]:
# 1 Make vector of zeros of the same length of the columns of Y
i_j = np.zeros(Y.shape[1])

# 2 Turn into 1's the 0s of the columns you don't want to analyse
i_j[:4] = 1

# 3 multiply the diagonlized i_j vector by Y 
#Y_r1 = Y@np.diag(i_j)

# we reapply the column labels that where lost by multiplying the numpy array by the dataframe
#Y_r1.columns = pd.MultiIndex.from_frame(mr_fd_labels)
 
selection_demand = idx[:,:]
selection_sectors = idx[:,"Services"]

# own method
Y_mask = Y*np.nan
Y_mask.loc[selection_sectors,selection_demand] = 1
Y_r1 = Y*Y_mask
Y_r1 = Y_r1.dropna(axis=1,how="all").replace(np.nan,0)

# Let's now check what the output looks like
Y_r1


f=F @ np.diag(1/x_out)
f.columns = x_out.index

F_star=f@L@Y_r1.values
F_star.columns = Y_r1.columns

#visualising data
F_tot = (F.loc[:,selection_sectors]).sum(axis=1)
result = np.diag(100/F_star.sum(axis=1))@F_star
result.index = F.index
result.columns = pd.MultiIndex.to_flat_index(result.columns)
result = result.sort_values("CO2 emissions (unit: tonnes/year)",axis=1,ascending=False).round()
result.loc[:,result.max(axis=0)>10]

,"(OECD, Final consumption expenditure by household)","(OECD, Final consumption expenditure by government)","(ROW, Final consumption expenditure by household)","(BRICS, Final consumption expenditure by household)","(BRICS, Final consumption expenditure by government)","(ROW, Final consumption expenditure by government)"
extension_name,,,,,,
CO2 emissions (unit: tonnes/year),23.0,13.0,12.0,11.0,10.0,8.0
Blue water consumption (unit: million m3/year),11.0,6.0,16.0,17.0,16.0,10.0
Employment (unit: 1000 people/year),15.0,8.0,20.0,11.0,10.0,12.0


However, if you want to identify the position of a specific category within a region you would need to find the exact column. 
That's not always obvious. 

If you know the exact order of regions and sectors, you can identify the position by performing the following procedure

In [17]:
# We want the final demand category in position 3 (i.e. gross fixed capital formation)
# remember in python we start counting at 0
category_position = 3

# However we are only interested in region 2 (i.e., Brics)
# remember in python we start counting at 0
region_position = 1

# we know that in total we have 4 final demand category
no_of_Y_categories = 4

# so the numerical position of gross fixed capital formation in the BRICS region is

Y_cat_position = region_position * no_of_Y_categories + category_position
Y_cat_position  

7

Now you can calculate your modified Y

In [18]:
# 1 Make vector of zeros of the same length of the columns of Y
i_j = np.zeros(Y.shape[1])

# 2 Turn into 1's the 0s of the columns you don't want to analyse
i_j[Y_cat_position] = 1

# 3 multiply the diagonlized i_j vector by Y 
Y_cat_3_reg_2 = Y@np.diag(i_j)

# we reapply the column labels that where lost by multiplying the numpy array by the dataframe
Y_cat_3_reg_2.columns = pd.MultiIndex.from_frame(mr_fd_labels)

Y_cat_3_reg_2

region                                                             OECD  \
final_demand_category        Final consumption expenditure by household   
region sector                                                             
OECD   Food                                                         0.0   
       Clothing                                                     0.0   
       Shelter                                                      0.0   
       Construction                                                 0.0   
       Manufactured products                                        0.0   
       Mobility                                                     0.0   
       Trade                                                        0.0   
       Services                                                     0.0   
BRICS  Food                                                         0.0   
       Clothing                                                     0.0   
       Shelter                                                      0.0   
       Construction                                                 0.0   
       Manufactured products                                        0.0   
       Mobility                                                     0.0   
       Trade                                                        0.0   
       Services                                                     0.0   
ROW    Food                                                         0.0   
       Clothing                                                     0.0   
       Shelter                                                      0.0   
       Construction                                                 0.0   
       Manufactured products                                        0.0   
       Mobility                                                     0.0   
       Trade                                                        0.0   
       Services                                                     0.0   

region                                                                \
final_demand_category        Final consumption expenditure by NPISHs   
region sector                                                          
OECD   Food                                                      0.0   
       Clothing                                                  0.0   
       Shelter                                                   0.0   
       Construction                                              0.0   
       Manufactured products                                     0.0   
       Mobility                                                  0.0   
       Trade                                                     0.0   
       Services                                                  0.0   
BRICS  Food                                                      0.0   
       Clothing                                                  0.0   
       Shelter                                                   0.0   
       Construction                                              0.0   
       Manufactured products                                     0.0   
       Mobility                                                  0.0   
       Trade                                                     0.0   
       Services                                                  0.0   
ROW    Food                                                      0.0   
       Clothing                                                  0.0   
       Shelter                                                   0.0   
       Construction                                              0.0   
       Manufactured products                                     0.0   
       Mobility                                                  0.0   
       Trade                                                     0.0   
       Services                                                  0.0   

region                       

#### 3.3 Slicing approach

In the slicing approach you take advantage of the slicing methods built within pandas and numpy

Therefore, if we wanted to select all OECD final demand categories we would execute the following

In [19]:
Y_oecd = Y.loc[:, "OECD"] 
Y_oecd

final_demand_category         Final consumption expenditure by household  \
region sector                                                              
OECD   Food                                                 1.753360e+06   
       Clothing                                             2.884869e+05   
       Shelter                                              6.622146e+05   
       Construction                                         6.848830e+04   
       Manufactured products                                1.786986e+06   
       Mobility                                             1.157830e+06   
       Trade                                                4.004337e+05   
       Services                                             1.070068e+07   
BRICS  Food                                                 2.223431e+04   
       Clothing                                             1.077132e+05   
       Shelter                                              1.453606e+04   
       Construction                                         3.443303e+03   
       Manufactured products                                1.644037e+05   
       Mobility                                             1.929819e+04   
       Trade                                                1.828862e+02   
       Services                                             2.200201e+04   
ROW    Food                                                 1.361017e+05   
       Clothing                                             7.658174e+04   
       Shelter                                              2.659790e+04   
       Construction                                         5.852935e+03   
       Manufactured products                                2.829310e+05   
       Mobility                                             4.871657e+04   
       Trade                                                5.448228e+02   
       Services                                             8.166793e+04   

final_demand_category         Final consumption expenditure by NPISHs  \
region sector                                                           
OECD   Food                                              1.610136e+03   
       Clothing                                          1.290528e+01   
       Shelter                                           1.939060e+03   
       Construction                                      1.349671e+02   
       Manufactured products                             4.629975e+03   
       Mobility                                          9.812754e+03   
       Trade                                             2.697264e+03   
       Services                                          2.968987e+06   
BRICS  Food                                              1.416137e+01   
       Clothing                                          2.482360e-04   
       Shelter                                           2.450880e+00   
       Construction                                      1.208981e-02   
       Manufactured products                             4.023717e+02   
       Mobility                                          2.489097e+01   
       Trade                                             3.124122e+02   
       Services                                          9.771105e+02   
ROW    Food                                              1.065831e+02   
       Clothing                                          7.118259e-03   
       Shelter                                           2.482202e+01   
       Construction                                      1.707257e-01   
       Manufactured products                             2.148106e+03   
       Mobility                                          2.460445e+02   
       Trade                                             9.219041e+02   
       Services                                          2.246062e+04   

final_demand_category         Final consumption expenditure by government  \
region sector                                  

N.b in this case we don't need to use pd.IndexSlice because OECD is at top level of the hierarchy of the multi-index

However if we needed to select the "Shelter" sector in the region we would need to use 

<code>Y.loc[:,pd.IndexSlice["BRICS", "Gross capital formation"]]</code>

because the sector is in this case is at a lower hierarchy level

In [20]:
Y_oecd_gfcf = Y.loc[:,pd.IndexSlice["BRICS", "Gross capital formation"]]
Y_oecd_gfcf

region  sector               
OECD    Food                     2.303218e+03
        Clothing                 1.605304e+02
        Shelter                  2.478947e+02
        Construction             5.940461e+03
        Manufactured products    2.028295e+05
        Mobility                 3.840281e+02
        Trade                    1.899609e+03
        Services                 3.957353e+03
BRICS   Food                     7.795218e+04
        Clothing                 1.838193e+04
        Shelter                  7.618184e+04
        Construction             2.474604e+06
        Manufactured products    1.728212e+06
        Mobility                 2.382546e+04
        Trade                    1.402123e+04
        Services                 3.518482e+05
ROW     Food                     2.982710e+03
        Clothing                 1.534636e+02
        Shelter                  1.679753e+03
        Construction             7.029510e+03
        Manufactured products    9.256903e+04
    

#### 3.4 Let's visualy compare the results of the two approaches

We see that they are the same

In [21]:
pd.concat([Y_cat_3_reg_2.sum(axis=1), Y_oecd_gfcf],axis=1)

0  (BRICS, Gross capital formation)
region sector                                                               
OECD   Food                   2.303218e+03                      2.303218e+03
       Clothing               1.605304e+02                      1.605304e+02
       Shelter                2.478947e+02                      2.478947e+02
       Construction           5.940461e+03                      5.940461e+03
       Manufactured products  2.028295e+05                      2.028295e+05
       Mobility               3.840281e+02                      3.840281e+02
       Trade                  1.899609e+03                      1.899609e+03
       Services               3.957353e+03                      3.957353e+03
BRICS  Food                   7.795218e+04                      7.795218e+04
       Clothing               1.838193e+04                      1.838193e+04
       Shelter                7.618184e+04                      7.618184e+04
       Construction           2.474604e+06                      2.474604e+06
       Manufactured products  1.728212e+06                      1.728212e+06
       Mobility               2.382546e+04                      2.382546e+04
       Trade                  1.402123e+04                      1.402123e+04
       Services               3.518482e+05                      3.518482e+05
ROW    Food                   2.982710e+03                      2.982710e+03
       Clothing               1.534636e+02                      1.534636e+02
       Shelter                1.679753e+03                      1.679753e+03
       Construction           7.029510e+03                      7.029510e+03
       Manufactured products  9.256903e+04                      9.256903e+04
       Mobility               5.840814e+02                      5.840814e+02
       Trade                  2.179709e+02                      2.179709e+02
       Services               1.527349e+03                      1.527349e+03

<font color="red">**Unless you are 100% sure of the cleanliness/quality of your labels, the mathematical approach is better**</font>

The mathematical approach is preferable for multiple reasons:

- it maintains the order (i.e., shape) of Y unalterd

- it's not affected by typos and inconsistencies in labels

- **it can be easily combined with the selection of one or more specific sector footprint such that: $\mathbf{Y^{reg}_{sec}}  = \mathrm{\hat{i_{i}^{sec}}} \mathbf{Y^{reg}}$**

- **it relies on the same principle for final demand changes**


#### 3.5 Selecting a specific product consumed in a region and applying changes in final demand

For example if we also wanted to isolate a product categoy

In [22]:
# 1 vector of zeros of the same length of the rows
i_i = np.zeros(Y.shape[0])

# 2 turn into 1's all the 0's that concerns rows (sectors) your want to analyse
# in our case we want to isolate Shelter in position 2
i_i[2] = 1

# 3 multiply the diagonlized i_i vector by the matrix Y_r1 
Y_r1_sec3 = np.diag(i_i) @ Y_r1

# we reapply the index labels that where lost by multiplying the numpy array by the dataframe
Y_r1_sec3.index = pd.MultiIndex.from_frame(mr_sec_labels)

Y_r1_sec3

region                                                             OECD  \
final_demand_category        Final consumption expenditure by household   
region sector                                                             
OECD   Food                                                        0.00   
       Clothing                                                    0.00   
       Shelter                                                662214.58   
       Construction                                                0.00   
       Manufactured products                                       0.00   
       Mobility                                                    0.00   
       Trade                                                       0.00   
       Services                                                    0.00   
BRICS  Food                                                        0.00   
       Clothing                                                    0.00   
       Shelter                                                     0.00   
       Construction                                                0.00   
       Manufactured products                                       0.00   
       Mobility                                                    0.00   
       Trade                                                       0.00   
       Services                                                    0.00   
ROW    Food                                                        0.00   
       Clothing                                                    0.00   
       Shelter                                                     0.00   
       Construction                                                0.00   
       Manufactured products                                       0.00   
       Mobility                                                    0.00   
       Trade                                                       0.00   
       Services                                                    0.00   

region                                                                \
final_demand_category        Final consumption expenditure by NPISHs   
region sector                                                          
OECD   Food                                                   0.0000   
       Clothing                                               0.0000   
       Shelter                                             1939.0603   
       Construction                                           0.0000   
       Manufactured products                                  0.0000   
       Mobility                                               0.0000   
       Trade                                                  0.0000   
       Services                                               0.0000   
BRICS  Food                                                   0.0000   
       Clothing                                               0.0000   
       Shelter                                                0.0000   
       Construction                                           0.0000   
       Manufactured products                                  0.0000   
       Mobility                                               0.0000   
       Trade                                                  0.0000   
       Services                                               0.0000   
ROW    Food                                                   0.0000   
       Clothing                                               0.0000   
       Shelter                                                0.0000   
       Construction                                           0.0000   
       Manufactured products                                  0.0000   
       Mobility                                               0.0000   
       Trade                                                  0.0000   
       Services                                               0.0000   

region                       

or if we wanted to analyse a 20% change for a specific category

In [23]:
# 1 vector of zeros of the same length of the rows
i_i = np.zeros(Y.shape[0])

# 2 turn into 0.2 (i.e., 20% ) all the 0's that concerns rows (sectors) your want to analyse
# in our case we want to isolate Shelter in position 2
i_i[2] = 0.2

# 3 multiply the diagonlized i_i vector by the matrix Y_r1 
delta_Y_r1_sec3 = np.diag(i_i) @ Y_r1

# we reapply the index labels that where lost by multiplying the numpy array by the dataframe
delta_Y_r1_sec3.index = pd.MultiIndex.from_frame(mr_sec_labels)

delta_Y_r1_sec3

region                                                             OECD  \
final_demand_category        Final consumption expenditure by household   
region sector                                                             
OECD   Food                                                       0.000   
       Clothing                                                   0.000   
       Shelter                                               132442.916   
       Construction                                               0.000   
       Manufactured products                                      0.000   
       Mobility                                                   0.000   
       Trade                                                      0.000   
       Services                                                   0.000   
BRICS  Food                                                       0.000   
       Clothing                                                   0.000   
       Shelter                                                    0.000   
       Construction                                               0.000   
       Manufactured products                                      0.000   
       Mobility                                                   0.000   
       Trade                                                      0.000   
       Services                                                   0.000   
ROW    Food                                                       0.000   
       Clothing                                                   0.000   
       Shelter                                                    0.000   
       Construction                                               0.000   
       Manufactured products                                      0.000   
       Mobility                                                   0.000   
       Trade                                                      0.000   
       Services                                                   0.000   

region                                                                \
final_demand_category        Final consumption expenditure by NPISHs   
region sector                                                          
OECD   Food                                                  0.00000   
       Clothing                                              0.00000   
       Shelter                                             387.81206   
       Construction                                          0.00000   
       Manufactured products                                 0.00000   
       Mobility                                              0.00000   
       Trade                                                 0.00000   
       Services                                              0.00000   
BRICS  Food                                                  0.00000   
       Clothing                                              0.00000   
       Shelter                                               0.00000   
       Construction                                          0.00000   
       Manufactured products                                 0.00000   
       Mobility                                              0.00000   
       Trade                                                 0.00000   
       Services                                              0.00000   
ROW    Food                                                  0.00000   
       Clothing                                              0.00000   
       Shelter                                               0.00000   
       Construction                                          0.00000   
       Manufactured products                                 0.00000   
       Mobility                                              0.00000   
       Trade                                                 0.00000   
       Services                                              0.00000   

region                       

## Exercise 4: Calculate Footprints

#### 4.1 Slice extension vectors 

In [24]:
# First we separate the Extensions into their respective vectors
e_CO2 = F[0]
e_water = F[1]
e_employment =F[2]

# We also collect the final demand extension for CO2 emissions
e_y_CO2 = F_y[0]
e_y_water = F_y[1]
# we don't collect employment in final demand because there is no employment to final demand

#### 4.2 Calculate the intensity vector of an extension

$\text{f} = \text{F} \hat{\text{X}}^{-1} $

In [25]:
# We then calculate the intensities of the extensions
f_CO2 = e_CO2 @ inv_diag_x

#### 4.3 Calculate the carbon footprint.

$\text{e} = \text{f} \mathbf{L}\text{Y} + \text{e}_{y}$

In [26]:
# Calculate the total global footprint
e_CO2_total = f_CO2 @ L @ Y.sum(axis=1) + e_y_CO2.sum()
e_CO2_total

34758162135.0

#### 4.4 Let's calculate the regional final demand breakdown

In [27]:
# Calculate the footprint broken down by region and final demand category
e_CO2_Y_breakdown = f_CO2 @ L @ Y
# we apply the columns labels
e_CO2_Y_breakdown = pd.DataFrame(e_CO2_Y_breakdown, index=pd.MultiIndex.from_frame(mr_fd_labels))
e_CO2_Y_breakdown

0
region final_demand_category                                    
OECD   Final consumption expenditure by household   6.195670e+09
       Final consumption expenditure by NPISHs      4.105011e+08
       Final consumption expenditure by government  9.452353e+08
       Gross capital formation                      2.940609e+09
BRICS  Final consumption expenditure by household   3.960359e+09
       Final consumption expenditure by NPISHs      6.854062e+08
       Final consumption expenditure by government  1.348961e+09
       Gross capital formation                      6.306992e+09
ROW    Final consumption expenditure by household   3.321607e+09
       Final consumption expenditure by NPISHs      3.145537e+08
       Final consumption expenditure by government  6.215356e+08
       Gross capital formation                      2.589339e+09

#### 4.5 Sum the regional values together to obtain the regional footprint. Do not forget to also add the the final demand extensions from F_y

In [28]:
# Calculate the total regional footprint
e_CO2_oecd = e_CO2_Y_breakdown.iloc[:4].sum() + e_y_CO2[0] # Remember that this vector only has 3 items referring to household of each region
e_CO2_brics = e_CO2_Y_breakdown.iloc[4:8].sum() + e_y_CO2[1]
e_CO2_row = e_CO2_Y_breakdown.iloc[8:].sum() + e_y_CO2[2]

# organize them into a dataframe
CO2_reg_footprint = pd.concat([e_CO2_oecd, e_CO2_brics, e_CO2_row], axis=1)
CO2_reg_footprint.columns = ["OECD", "BRICS", "ROW"]
CO2_reg_footprint.index = [f_lb[0]]
CO2_reg_footprint

,OECD,BRICS,ROW
CO2 emissions (unit: tonnes/year),1.313563e+10,1.335968e+10,8.262854e+09


## Exercise 5: Sectoral contribution to the carbon footprint of a region

$\text{F}_s = \hat{\text{f}} \mathbf{L} \text{Y}$

#### 5.1 First we make our final demand for the BRICS region

In [29]:
# 1 Make vector of ones of the same length of the columns of Y
i_j = np.zeros(Y.shape[1])

# 2 Turn into 0's the 1s of the columns you don't want to analyse
i_j[4:8] = 1

# 3 multiply the diagonlized i_j vector by Y 
Y_brics = Y@np.diag(i_j)

# we reapply the column labels that where lost by multiplying the numpy array by the dataframe
Y_brics.columns = pd.MultiIndex.from_frame(mr_fd_labels)

# Let's now check what the output looks like 
Y_brics

region                                                             OECD  \
final_demand_category        Final consumption expenditure by household   
region sector                                                             
OECD   Food                                                         0.0   
       Clothing                                                     0.0   
       Shelter                                                      0.0   
       Construction                                                 0.0   
       Manufactured products                                        0.0   
       Mobility                                                     0.0   
       Trade                                                        0.0   
       Services                                                     0.0   
BRICS  Food                                                         0.0   
       Clothing                                                     0.0   
       Shelter                                                      0.0   
       Construction                                                 0.0   
       Manufactured products                                        0.0   
       Mobility                                                     0.0   
       Trade                                                        0.0   
       Services                                                     0.0   
ROW    Food                                                         0.0   
       Clothing                                                     0.0   
       Shelter                                                      0.0   
       Construction                                                 0.0   
       Manufactured products                                        0.0   
       Mobility                                                     0.0   
       Trade                                                        0.0   
       Services                                                     0.0   

region                                                                \
final_demand_category        Final consumption expenditure by NPISHs   
region sector                                                          
OECD   Food                                                      0.0   
       Clothing                                                  0.0   
       Shelter                                                   0.0   
       Construction                                              0.0   
       Manufactured products                                     0.0   
       Mobility                                                  0.0   
       Trade                                                     0.0   
       Services                                                  0.0   
BRICS  Food                                                      0.0   
       Clothing                                                  0.0   
       Shelter                                                   0.0   
       Construction                                              0.0   
       Manufactured products                                     0.0   
       Mobility                                                  0.0   
       Trade                                                     0.0   
       Services                                                  0.0   
ROW    Food                                                      0.0   
       Clothing                                                  0.0   
       Shelter                                                   0.0   
       Construction                                              0.0   
       Manufactured products                                     0.0   
       Mobility                                                  0.0   
       Trade                                                     0.0   
       Services                                                  0.0   

region                       

#### 5.2 Calculate the footprint by diagonalizing the intensity vector

In [30]:
# In this case we diagonalize the emission intensity vector 
e_brics_s = np.diag(f_CO2) @ L @ Y_brics.sum(axis=1)
e_brics_s =  pd.DataFrame(e_brics_s, index=pd.MultiIndex.from_frame(mr_sec_labels), columns=[f_lb[0]])
e_brics_s

CO2 emissions (unit: tonnes/year)
region sector                                                  
OECD   Food                                        1.345596e+07
       Clothing                                    2.303452e+06
       Shelter                                     1.518169e+08
       Construction                                9.973621e+06
       Manufactured products                       7.641130e+07
       Mobility                                    5.910159e+07
       Trade                                       2.803610e+06
       Services                                    1.105158e+07
BRICS  Food                                        3.206021e+08
       Clothing                                    4.080431e+07
       Shelter                                     5.546750e+09
       Construction                                2.261010e+09
       Manufactured products                       1.957440e+09
       Mobility                                    5.678729e+08
       Trade                                       2.515356e+07
       Services                                    4.296644e+08
ROW    Food                                        3.145549e+07
       Clothing                                    1.287554e+07
       Shelter                                     5.563897e+08
       Construction                                5.049048e+07
       Manufactured products                       1.023447e+08
       Mobility                                    5.763203e+07
       Trade                                       4.745642e+06
       Services                                    9.568654e+06

## Exercise 6: Calculate the regional footprint of each extension
Independent exercise